<a href="https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_income/02_bullet_bonds.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Bullet Bonds

## Learning Objectives

- Define the cash flows of a bullet bond. 
- Understand how the price of a bond is quoted.
- Calculating bond price from the discount rate. 
- Learn about duration and convexity of a bullet bond. 



## Cash Flow Structure

A bullet bond returns the principal at maturity. This is an example of a bullet bond and its cash flows:

In [ ]:
import pandas as pd
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import IPython.display as display

# Define the bond
bond = FixedRateBullet(
    issue_dt="2026-01-01",
    maturity="2031-01-01",
    cpn=5,               # Coupon rate (%)
    cpn_freq=1,          # Annual coupon
    notional=100,        # Face value
    yield_to_maturity=0.05,  # Input YTM
    settlement_date="2027-01-01",
)

# Build table with only inputs
inputs_data = [
    {"Attribute": "Instrument", "Value": bond.__class__.__name__},
    {"Attribute": "Issue Date", "Value": bond.issue_dt},
    {"Attribute": "Maturity Date", "Value": bond.maturity},
    {"Attribute": "Settlement Date", "Value": bond.get_settlement_date()},
    {"Attribute": "Notional", "Value": bond.notional},
    {"Attribute": "Coupon (%)", "Value": bond.cpn},
    {"Attribute": "Coupon Frequency", "Value": bond.cpn_freq},
    {"Attribute": "Day Count Convention", "Value": bond.day_count_convention.__class__.__name__},
    {"Attribute": "Yield to Maturity (%)", "Value": f"{bond.yield_to_maturity() * 100:.2f}"},
]

df_inputs = pd.DataFrame(inputs_data)

# Display table without index
display.display(df_inputs.style.hide(axis="index"))


,Attribute,Value
0,Instrument,FixedRateBullet
1,Issue Date,2020-01-01 00:00:00
2,Maturity Date,2030-01-01 00:00:00
3,Settlement Date,None
4,Notional,100
5,Coupon (%),5
6,Coupon Frequency,1
7,Day Count Convention,DayCount30365


The cash flow structure is:

In [36]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import pandas as pd
from IPython.display import display

df = pd.DataFrame(bond.cash_flows())
df.columns = ["Value"]
display(df.style.hide(axis="index"))

Value
5.000000
5.000000
5.000000
105.000000


## Calculating Bond Price from Discount Curve

In order to price a bond correctly, we just need to compute the present value of the future cash flows. The cash flows consist of the periodic coupon payments and the face value paid at maturity, in the case of the bullet bond. In order to discount the cash flows we need to find the discount rate, which is found using the exponential function: $P = F \times e^{-rt}$.

## Duration

Duration measures a bond's price sensitivity to changes in interest rates and represents the weighted average time to receive the bond's cash flows. Mathematically, duration approximates the percentage change in a bond's price for a small change in yield, expressed as:

$$D = -\frac{1}{P} \cdot \frac{dP}{dr}$$

The negative sign reflects the inverse relationship between bond prices and yields.

### Macaulay Duration

Macaulay duration is the weighted average time until a bond's cash flows are received, expressed in years. It is calculated as:

$$\frac{\sum_{t=1}^n \frac{t \cdot C_t}{(1 + y)^t} + \frac{n \cdot FV}{(1 + y)^n}}{P}$$

where:
- $C_t$: Cash flow (coupon payment) at time $t$
- $FV$: Face value of the bond
- $y$: Yield to maturity (per period)
- $n$: Number of periods until maturity
- $P$: Present value (current price) of the bond

For a zero-coupon bond, since there are no coupon payments ($C_t = 0$), the Macaulay duration simplifies to the time to maturity, $T - t$, where $T$ is the maturity date and $t$ is the current date.

### Modified Duration

Modified duration directly measures the percentage change in a bond's price for a 1% change in yield. It is derived from the Macaulay duration as:

$$D^{\text{Mod}} = \frac{D^{\text{MC}}}{1 + \frac{y}{k}}$$

where:
- $y$: Annual yield to maturity
- $k$: Number of compounding periods per year (e.g., $k = 2$ for semi-annual, $k = 1$ for annual)

For a zero-coupon bond with annual compounding ($k = 1$), the modified duration is:

$$D^{\text{Mod}} = \frac{T - t}{1 + y}$$



## Convexity

Convexity measures the sensitivity of a bond’s duration to changes in interest rates and captures the non-linear relationship between bond prices and yields. It is the second derivative of the bond’s price with respect to the yield, scaled by the price:

$$C = \frac{1}{P} \cdot \frac{d^2P}{dr^2}$$

For a zero-coupon bond with annual compounding, the convexity is:

$$C = \frac{(T - t)(T - t + 1)}{(1 + y)^2}$$

In [ ]:
import pandas as pd
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import IPython.display as display

# Calculate outputs
price = bond.get_price()
ytm = bond.get_yield_to_maturity()
macaulay_duration = bond.macaulay_duration()
modified_duration = bond.modified_duration()
convexity = bond.convexity()  # Convexity calculation

# Build table
data = [
    # Inputs
    {"Attribute": "Instrument", "Value": bond.__class__.__name__, "Category": ""},
    {"Attribute": "Issue Date", "Value": bond.issue_dt, "Category": ""},
    {"Attribute": "Maturity Date", "Value": bond.maturity, "Category": ""},
    {"Attribute": "Settlement Date", "Value": bond.get_settlement_date(), "Category": ""},
    {"Attribute": "Notional", "Value": bond.notional, "Category": ""},
    {"Attribute": "Coupon (%)", "Value": bond.cpn, "Category": ""},
    {"Attribute": "Coupon Frequency", "Value": bond.cpn_freq, "Category": ""},
    {"Attribute": "Day Count Convention", "Value": bond.day_count_convention.__class__.__name__, "Category": ""},
    {"Attribute": "Yield to Maturity (%) (Input)", "Value": f"{bond.yield_to_maturity() * 100:.2f}", "Category": ""},
    
    # Calculated
    {"Attribute": "Price", "Value": f"{price:.2f}", "Category": "★"},
    {"Attribute": "Yield to Maturity (%)", "Value": f"{ytm * 100:.2f}", "Category": "★"},
    {"Attribute": "Macaulay Duration (Years)", "Value": f"{macaulay_duration:.2f}", "Category": "★"},
    {"Attribute": "Modified Duration (Years)", "Value": f"{modified_duration:.2f}", "Category": "★"},
    {"Attribute": "Convexity", "Value": f"{convexity:.4f}", "Category": "★"},
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Display table without index
display.display(df.style.hide(axis="index"))


Attribute,Value,Category
Instrument,FixedRateBullet,
Issue Date,2026-01-01 00:00:00,
Maturity Date,2031-01-01 00:00:00,
Settlement Date,2027-01-01 00:00:00,
Notional,100,
Coupon (%),5,
Coupon Frequency,1,
Day Count Convention,DayCountActualActualBond,
Yield to Maturity (%) (Input),5.00,
Price,99.78,★
